In [1]:
import os
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

# Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [3]:
df = pd.read_csv("../data-total.csv")
df.columns
# rename columns
df.rename(columns={ 'Lat': 'lat', 
                    'Long': 'long', 
                    'Satellite': 'satellite', 
                    'Time_UTC': 'time_utc', 
                    'Date': 'date', 
                    'Source': 'source', 
                    'PROVINSI': 'provinsi',
                    'KAB_KOTA': 'kab_kota'}, inplace=True)
# adding '-' to date column values
df.date = pd.to_datetime(df.date, format="%Y%m%d")
df.date = df.date.astype('string')

In [4]:
df.head()

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw
0,-2.977430,112.868,LPN-NPP,05:50:44 AM,2016-08-25,LAPAN,KALIMANTAN TENGAH,KOTAWARINGIN TIMUR,MENTAYA HILIR SELATAN,SAMUDA BESAR,NaN
1,-5.062590,105.101,LPN-NPP,06:11:04 AM,2016-08-24,LAPAN,LAMPUNG,LAMPUNG TENGAH,ANAK TUHA,BUMI JAYA,NaN
2,-8.293230,130.812,LPN-NPP,04:30:05 AM,2016-08-24,LAPAN,MALUKU,MALUKU TENGGARA BARAT,TANIMBAR SELATAN DAYA,WERAIN,NaN
3,-0.512625,110.168,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,KALIMANTAN BARAT,KETAPANG,SIMPANG HULU,SEKUCING LABAI,NaN
4,1.189910,100.672,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HULU,KECAMATAN KAPENUHAN,UPT SP IV KOTO TENGAH,NaN


In [5]:
len(df)

39728

In [6]:
import ee
import folium
from folium import plugins

In [8]:
# Authenticate earth engine servers
ee.Authenticate()
#initialize API
ee.Initialize()

Enter verification code: 4/1AX4XfWjnp3Sq15KUW9EWkuEJ8AvkNXKu3wyMtv0mLunmHCCeiPbGtcSUSXo

Successfully saved authorization token.


In [9]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.ImageCollection(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [10]:
#################################################################################################################################
# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\
#################################################################################################################################


####################################################### SENTINEL ################################################################
# Select row column
row = 5001


# Set coordinates
coordinates = [float(df.lat.values[row]), float(df.long.values[row])]
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# filter image collection
# set base date
base_date = ee.Date(df.date.values[row])

#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# sentinel image pre wildfire event
pre_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()


# Calculate NBR for pre fire
pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# sentinel image post wildfire event
post_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()
            

# Calculate NBR for post fire
post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=df.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# os.chdir(file_path)
import urllib

download_url = post_sentinel_collection.getDownloadURL({
    'region': region,
    'bands' : ['DELTA_NBR'],
    'scale' : 50,
    'format': 'GeoTIFF'
})

# download_path = os.path.join(file_path)
print(f"download data from {download_url}")
urllib.request.urlretrieve(download_url,'b9.tif')
print(f"{download_url} downloaded")
# import urllib
# file_path = "C:\\Users\\USER\\Downloads\\aa"
# urllib.request.urlretrieve(download_url, file_path)

download data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/726eaaf14ef5d88dbf47fc032e258477-6a6c43fd3ce7a0523da30c66213f2ae6:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/726eaaf14ef5d88dbf47fc032e258477-6a6c43fd3ce7a0523da30c66213f2ae6:getPixels downloaded
